<a href="https://colab.research.google.com/github/SiddhiMane/Sem-8/blob/main/HPC_Prac2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%writefile parallel_sort.cpp
#include <iostream>
#include <vector>
#include <omp.h>

using namespace std;

// Parallel Bubble Sort
void parallelBubbleSort(vector<int>& arr) {
    int n = arr.size();
    bool swapped;

    #pragma omp parallel shared(arr, swapped)
    {
        do {
            swapped = false;

            #pragma omp for
            for (int i = 0; i < n-1; ++i) {
                if (arr[i] > arr[i+1]) {
                    swap(arr[i], arr[i+1]);
                    swapped = true;
                }
            }
        } while (swapped);
    }
}

// Merge two sorted sub-arrays into one sorted array
void merge(vector<int>& arr, int l, int m, int r) {
    int n1 = m - l + 1;
    int n2 = r - m;

    vector<int> L(n1), R(n2);

    for (int i = 0; i < n1; ++i)
        L[i] = arr[l + i];
    for (int j = 0; j < n2; ++j)
        R[j] = arr[m + 1 + j];

    int i = 0, j = 0, k = l;

    while (i < n1 && j < n2) {
        if (L[i] <= R[j]) {
            arr[k] = L[i];
            ++i;
        } else {
            arr[k] = R[j];
            ++j;
        }
        ++k;
    }

    while (i < n1) {
        arr[k] = L[i];
        ++i;
        ++k;
    }

    while (j < n2) {
        arr[k] = R[j];
        ++j;
        ++k;
    }
}

// Parallel Merge Sort
void parallelMergeSort(vector<int>& arr, int l, int r) {
    if (l < r) {
        int m = l + (r - l) / 2;

        #pragma omp parallel sections
        {
            #pragma omp section
            parallelMergeSort(arr, l, m);
            #pragma omp section
            parallelMergeSort(arr, m+1, r);
        }

        merge(arr, l, m, r);
    }
}

int main() {
    vector<int> arr = {12, 11, 13, 5, 6, 7};

    cout << "Original array: ";
    for (int num : arr)
        cout << num << " ";
    cout << endl;

    // Parallel Bubble Sort
    parallelBubbleSort(arr);
    cout << "Array after parallel bubble sort: ";
    for (int num : arr)
        cout << num << " ";
    cout << endl;

    // Resetting array for merge sort
    arr = {12, 11, 13, 5, 6, 7};

    // Parallel Merge Sort
    parallelMergeSort(arr, 0, arr.size() - 1);
    cout << "Array after parallel merge sort: ";
    for (int num : arr)
        cout << num << " ";
    cout << endl;

    return 0;
}


Writing parallel_sort.cpp


In [5]:
!g++ -fopenmp -o parallel_sort parallel_sort.cpp
!./parallel_sort


Original array: 12 11 13 5 6 7 
Array after parallel bubble sort: 5 6 7 11 12 13 
Array after parallel merge sort: 5 6 7 11 12 13 
